# Explore Parliament Speeches

1. remove duplicates
2. handle missings
3.. procedural or otherwise nonimportant speeches? - DONE

In [1]:
import pandas as pd

### Import complete dataset of speeches

In [39]:
data = pd.read_csv("../data/complete_data.csv")

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [40]:
len(data)

44716

In [3]:
data.head()

,speech_id,speaker_id,speech_text,legislative_period,protocol_nr,agenda_item_number,speakerId,firstName,lastName,party,fraction
0,f7f58b13-85f3-424a-6864-08da102a68d8,11001235,"Herr Präsident, ich nehme die Wahl an.\nDann b...",19,1,6,11001235,Wolfgang,Kubicki,FDP,FDP
1,d0c84378-49ab-47ce-61aa-08da102a68d8,11001938,Das ist der Fall. – Ich sehe keine weiteren Vo...,19,1,3,11001938,Dr. Wolfgang,Schäuble,CDU,NaN
2,9aff56c3-41cd-4e6b-633b-08da0f22a008,11002190,"Guten Morgen, liebe Kolleginnen und Kollegen! ...",19,1,1,11002190,Alterspräsident Dr. Hermann,Otto Solms,FDP,NaN
3,05ebe489-0cff-4575-661e-08da0f22a008,11002190,Die unterbrochene Sitzung ist wieder eröffnet....,19,1,6,11002190,Alterspräsident Dr. Hermann,Otto Solms,FDP,NaN
4,901e2196-b575-4fea-5fd2-08da102a68d8,11002190,Herr Bundespräsident! Verehrte Kolleginnen und...,19,1,4,11002190,Alterspräsident Dr. Hermann,Otto Solms,FDP,NaN


In [4]:
import string
# defining normalization of text: converting to lowercase and removing punctuation
def normalize_text(input_str:str):
    return input_str.lower().translate(str.maketrans('','', string.punctuation))


### Create some helper columns for identification of speeches to include

In [5]:
# normalize (remove punctuation, to lower)
data['normalized'] = data['speech_text'].apply(normalize_text)
# Tokenized speech
data['tokenized'] = data['normalized'].str.split()
# length of speech
data['speech_length'] = data['tokenized'].apply(len)

### Initial inspection

In [6]:
tokenized = data['tokenized'].dropna()

# 1. Total number of speeches
num_speeches = len(tokenized)

# 2. Average speech length (in words)
avg_length = tokenized.apply(len).mean()

# 3. Maximum and minimum speech length
max_length = tokenized.apply(len).max()
min_length = tokenized.apply(len).min()

# 4. Vocabulary size (unique words across all utterances)
all_words = [word for sublist in tokenized for word in sublist]
vocab_size = len(set(all_words))

# 5. Total word count
total_words = len(all_words)

# Show results
print(f"Total number of speech: {num_speeches}")
print(f"Average speech length (words): {avg_length:.2f}")
print(f"Max speech length: {max_length} words")
print(f"Min speech length: {min_length} words")
print(f"Total word count: {total_words}")
print(f"Vocabulary size: {vocab_size}")


Total number of speech: 44716
Average speech length (words): 563.07
Max speech length: 9349 words
Min speech length: 1 words
Total word count: 25178388
Vocabulary size: 333593


### Duplicates?

In [7]:
print(len(data['speech_id'].unique())) # unique speech identifier
print(len(data))

44716
44716


-> no duplicates

### Missings?

In [8]:
na_counts = data.isna().sum()
na_counts = na_counts[na_counts > 0]
print(na_counts)

party         60
fraction    4414
dtype: int64


-> keep only rows with party info (30 rows without party info are no relevant loss to the dataset), fraction information won't be needed

In [9]:
data = data[data['party'].notna()].reset_index(drop=True)

### Importing meta information to speeches

In [10]:
df_agenda = pd.read_csv("../data/agenda_item_info.csv")

### Collection of Speeches to be excluded

#### identify via keywords in agenda item title

In [11]:
contains_matches = df_agenda['AgendaItemTitle'].str.contains(
    "Wahl|Eröffnung|Eidesleistung|Bekanntgabe|Fragestunde|Befragung|Nationalhymne|Geschäftsordnung", na=False # formal / procedural content
)

# filter mask
mask = contains_matches

# store matches in new df
to_drop = df_agenda.loc[mask, [
    'AgendaItemTitle', 
    'LegislaturePeriod', 
    'Number', 
    'AgendaItemNumber', 
    'Order', 
    'DateOnly'
]].rename(columns={
    'LegislaturePeriod': 'Legislature',
    'Number': 'Protocol_Nr'
}).reset_index(drop=True)


In [12]:
to_drop

,AgendaItemTitle,Legislature,Protocol_Nr,AgendaItemNumber,Order,DateOnly
0,Eröffnung der Sitzung durch den Alterspräsidenten,19,1,1,1,2017-10-24
1,Beschlussfassung über die Geschäftsordnung,19,1,2,2,2017-10-24
2,Wahl des Präsidenten,19,1,3,3,2017-10-24
3,Wahl der Stellvertreter des Präsidenten,19,1,6,6,2017-10-24
4,Nationalhymne,19,1,7,7,2017-10-24
...,...,...,...,...,...,...
375,Befragung der Bundesregierung AA und BMBF,20,96,1,1,2023-04-19
376,Fragestunde,20,96,2,2,2023-04-19
377,Wahlen zu Gremien,20,97,"11, 12",7,2023-04-20
378,Befragung der Bundesregierung BMJ und BMUV,20,99,1,1,2023-04-26


#### match titles via lagislature, protocol and agendaitemnumber/order to speech content

In [13]:
# MultiIndex-Mask
mask = data.set_index(['legislative_period', 'protocol_nr', 'agenda_item_number']).index.isin(
    to_drop.set_index(['Legislature', 'Protocol_Nr', 'Order']).index
)

matched_data = data[mask].reset_index(drop=True)


In [14]:
matched_data

,speech_id,speaker_id,speech_text,legislative_period,protocol_nr,agenda_item_number,speakerId,firstName,lastName,party,fraction,normalized,tokenized,speech_length
0,f7f58b13-85f3-424a-6864-08da102a68d8,11001235,"Herr Präsident, ich nehme die Wahl an.\nDann b...",19,1,6,11001235,Wolfgang,Kubicki,FDP,FDP,herr präsident ich nehme die wahl an\ndann beg...,"[herr, präsident, ich, nehme, die, wahl, an, d...",22
1,d0c84378-49ab-47ce-61aa-08da102a68d8,11001938,Das ist der Fall. – Ich sehe keine weiteren Vo...,19,1,3,11001938,Dr. Wolfgang,Schäuble,CDU,NaN,das ist der fall – ich sehe keine weiteren vor...,"[das, ist, der, fall, –, ich, sehe, keine, wei...",572
2,9aff56c3-41cd-4e6b-633b-08da0f22a008,11002190,"Guten Morgen, liebe Kolleginnen und Kollegen! ...",19,1,1,11002190,Alterspräsident Dr. Hermann,Otto Solms,FDP,NaN,guten morgen liebe kolleginnen und kollegen ne...,"[guten, morgen, liebe, kolleginnen, und, kolle...",273
3,05ebe489-0cff-4575-661e-08da0f22a008,11002190,Die unterbrochene Sitzung ist wieder eröffnet....,19,1,6,11002190,Alterspräsident Dr. Hermann,Otto Solms,FDP,NaN,die unterbrochene sitzung ist wieder eröffnet\...,"[die, unterbrochene, sitzung, ist, wieder, erö...",257
4,917c3517-5bc1-4763-5c38-08da0f22a008,11003124,"Herr Präsident, ich nehme die Wahl an.\nDann b...",19,1,6,11003124,Hans-Peter,Friedrich,CSU,CDU/CSU,herr präsident ich nehme die wahl an\ndann beg...,"[herr, präsident, ich, nehme, die, wahl, an, d...",14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6347,ce70b8e4-855f-4c28-855e-c050c374df2e,11005115,Frau Präsidentin! Sehr geehrte Frau Ministerin...,20,99,1,11005115,Dunja,Kreiser,SPD,SPD,frau präsidentin sehr geehrte frau ministerin ...,"[frau, präsidentin, sehr, geehrte, frau, minis...",147
6348,838fe67e-51a1-4d26-acb7-258d5ede5972,11005132,"Vielen Dank, Frau Präsidentin. – Liebe Kollegi...",20,99,1,11005132,Helge,Limburg,BÜNDNIS 90/DIE GRÜNEN,Bündnis 90 / Die Grünen,vielen dank frau präsidentin – liebe kolleginn...,"[vielen, dank, frau, präsidentin, –, liebe, ko...",168
6349,9b6ed8ce-866c-4457-8527-35dce1891aa9,11005142,Vielen Dank. – Unser Koalitionsvertrag sieht e...,20,99,1,11005142,Zanda,Martens,SPD,SPD,vielen dank – unser koalitionsvertrag sieht eb...,"[vielen, dank, –, unser, koalitionsvertrag, si...",116
6350,df4836be-313b-4a08-ab1a-b225c7b0a5f0,11005142,"Sehr gerne. Vielen Dank, Frau Präsidentin. – M...",20,99,1,11005142,Zanda,Martens,SPD,SPD,sehr gerne vielen dank frau präsidentin – mein...,"[sehr, gerne, vielen, dank, frau, präsidentin,...",110


In [15]:
# how many rows affected?
data.value_counts(['legislative_period', 'protocol_nr', 'agenda_item_number']).reset_index(name='count').sort_values('count', ascending=False)


,legislative_period,protocol_nr,agenda_item_number,count
0,20,108,1,92
1,20,153,1,91
2,20,177,2,88
3,20,168,1,87
4,20,133,1,82
...,...,...,...,...
3771,19,115,25,1
3772,20,181,19,1
3773,19,107,32,1
3774,19,230,29,1


In [41]:
mask = ~data.set_index(['legislative_period', 'protocol_nr', 'agenda_item_number']).index.isin(
    to_drop.set_index(['Legislature', 'Protocol_Nr', 'Order']).index
)

# Gefiltertes DataFrame
data_cleaned = data[mask].reset_index(drop=True)

In [42]:
data_cleaned

,speech_id,speaker_id,speech_text,legislative_period,protocol_nr,agenda_item_number,speakerId,firstName,lastName,party,fraction
0,901e2196-b575-4fea-5fd2-08da102a68d8,11002190,Herr Bundespräsident! Verehrte Kolleginnen und...,19,1,4,11002190,Alterspräsident Dr. Hermann,Otto Solms,FDP,NaN
1,a914bfea-8684-4446-0640-08da0f05b642,11002738,Herr Präsident! Kolleginnen und Kollegen! Die ...,19,10,3,11002738,Hans,Michelbach,CSU,CDU/CSU
2,02a4928b-8f4e-45d8-5f3e-08da102a68d8,11003422,Sehr geehrter Herr Präsident! Liebe Kolleginne...,19,10,3,11003422,Ingrid,Arndt-Brauer,SPD,SPD
3,ef59b72f-b33d-43da-5de1-08da0f22a008,11003646,Herr Präsident! Liebe Kolleginnen und Kollegen...,19,10,3,11003646,Antje,Tillmann,CDU,CDU/CSU
4,8dd9b261-a22c-431b-279d-08da0d9e2391,11003837,Herr Präsident! Liebe Kolleginnen und Kollegen...,19,10,3,11003837,Gerhard,Schick,BÜNDNIS 90/DIE GRÜNEN,Bündnis 90 / Die Grünen
...,...,...,...,...,...,...,...,...,...,...,...
38350,378c969e-0c23-4a4b-b359-9616d96735f7,11005221,Sehr geehrter Herr Präsident! Liebe Kolleginne...,20,99,5,11005221,Lina,Seitzl,SPD,SPD
38351,53f1bd4a-5aa5-4006-91a8-ceafe58a9de6,11005228,Frau Präsidentin! Meine Damen und Herren! Sie ...,20,99,3,11005228,Till,Steffen,BÜNDNIS 90/DIE GRÜNEN,Bündnis 90 / Die Grünen
38352,b4de6b5b-082e-45ac-ba6e-4403a82de283,11005264,Sehr geehrter Herr Präsident! Meine Damen und ...,20,99,4,11005264,Joachim,Wundrak,AfD,AfD
38353,dae72d62-2ab2-439a-9332-d585dcb35e54,11005289,Sehr geehrte Präsidentin! Liebe Kolleginnen un...,20,99,7,11005289,Clara,Bünger,DIE LINKE.,Die Linke


In [43]:
data_cleaned = data_cleaned[data_cleaned['speech_length'] > 200]
data_cleaned = data_cleaned[data_cleaned['speech_length'] < 1501]

KeyError: 'speech_length'

In [ ]:
len(data_cleaned)

### remove speeches by unobserved parties

In [ ]:
data_cleaned['party'].value_counts()

party
SPD                      8032
CDU                      6838
BÜNDNIS 90/DIE GRÜNEN    5067
AfD                      5041
FDP                      4683
DIE LINKE.               3618
CSU                      2838
Plos                      284
LKR                        82
Die PARTEI                 30
SSW                        20
Name: count, dtype: int64

In [20]:
parties_to_remove = ['Plos', 'LKR', 'Die PARTEI', 'SSW']

data_cleaned = data_cleaned[~data_cleaned['party'].isin(parties_to_remove)].reset_index(drop=True)

In [21]:
data_cleaned['party'].value_counts()

party
SPD                      8032
CDU                      6838
BÜNDNIS 90/DIE GRÜNEN    5067
AfD                      5041
FDP                      4683
DIE LINKE.               3618
CSU                      2838
Name: count, dtype: int64

In [22]:
tokenized = data_cleaned['tokenized'].dropna()

# 1. Total number of speeches
num_speeches = len(tokenized)

# 2. Average speech length (in words)
avg_length = tokenized.apply(len).mean()
med_length = tokenized.apply(len).median()
mod_length = tokenized.apply(len).mode()


# 3. Maximum and minimum speech length
max_length = tokenized.apply(len).max()
min_length = tokenized.apply(len).min()

# 4. Vocabulary size (unique words across all utterances)
all_words = [word for sublist in tokenized for word in sublist]
vocab_size = len(set(all_words))

# 5. Total word count
total_words = len(all_words)

# Show results
print(f"Total number of speech: {num_speeches}")
print(f"Average speech length (mean): {avg_length:.2f}")
print(f"Average speech length (median): {med_length:.2f}")
print(f"Average speech length (mode): {mod_length.iloc[0]:.2f}")
print(f"Max speech length: {max_length} words")
print(f"Min speech length: {min_length} words")
print(f"Total word count: {total_words}")
print(f"Vocabulary size: {vocab_size}")


Total number of speech: 36117
Average speech length (mean): 633.63
Average speech length (median): 607.00
Average speech length (mode): 584.00
Max speech length: 1500 words
Min speech length: 201 words
Total word count: 22884947
Vocabulary size: 319779


### formatting

In [23]:
df_agenda.columns

Index(['Unnamed: 0', 'AgendaItemTitle', 'AgendaItemNumber', 'Order',
       'AgendaItemDate', 'ProtocolId', 'DateOnly', 'ProtocolDate',
       'LegislaturePeriod', 'Number', 'SessionTitle', 'AgendaItemsCount',
       'MongoId', 'Id'],
      dtype='object')

In [24]:
data_cleaned.columns

Index(['speech_id', 'speaker_id', 'speech_text', 'legislative_period',
       'protocol_nr', 'agenda_item_number', 'speakerId', 'firstName',
       'lastName', 'party', 'fraction', 'normalized', 'tokenized',
       'speech_length'],
      dtype='object')

In [25]:
df_agenda = df_agenda.rename(columns={
    'LegislaturePeriod': 'legislative_period',
    'Number': 'protocol_nr',
    'Order':'agenda_item_number',
    'DateOnly':'date',
    'AgendaItemTitle':'agenda_item_title'
})

In [26]:
data_merged = data_cleaned.merge(
    df_agenda[['legislative_period', 'protocol_nr', 'agenda_item_number', 'agenda_item_title','date']],
    on=['legislative_period', 'protocol_nr', 'agenda_item_number'],
    how='left'
)


#### combine CDU/CSU, clean party names

In [27]:
data_merged['party'].value_counts()

party
SPD                      8032
CDU                      6838
BÜNDNIS 90/DIE GRÜNEN    5067
AfD                      5041
FDP                      4683
DIE LINKE.               3618
CSU                      2838
Name: count, dtype: int64

In [28]:
data_merged['party'] = data_merged['party'].replace(['CDU', 'CSU'], 'CDU/CSU')
data_merged['party'] = data_merged['party'].replace(['BÜNDNIS 90/DIE GRÜNEN'], 'GRÜNE')
data_merged['party'] = data_merged['party'].replace(['DIE LINKE.'], 'LINKE')

In [29]:
data_merged['party'].value_counts()

party
CDU/CSU    9676
SPD        8032
GRÜNE      5067
AfD        5041
FDP        4683
LINKE      3618
Name: count, dtype: int64

#### combine Names

In [30]:
data_merged['full_name'] = data_merged['firstName'] + ' ' + data_merged['lastName']


#### drop helper columns

In [31]:
data_merged = data_merged.drop(columns=['speech_id','speaker_id','speakerId','fraction','normalized','tokenized','speech_length','firstName','lastName'])

In [37]:
data_merged.to_csv("../data/final_data.csv")